In [1]:
import numpy as np
import pandas as pd
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
import astropy.units as u

## Предобработка таблицы

Открываю файл с каталогом

In [2]:
df = pd.read_csv('./A0V_list_filtered.txt', sep='\s+', skipinitialspace=True,
                 names=['name', 'RA', 'Dec', 'epoch', 'mag'])

Звёздную величину делаю численным значением

In [4]:
df['Vmag'] = df['mag'].map(lambda x: float(x.split('=')[-1]))
df = df.astype({'name': "string"})
# df

## Настройка запроса в симбад

In [6]:
cS = Simbad()

In [10]:
cS.add_votable_fields('flux(U)','flux(B)','flux(V)','flux(R)','flux(I)','flux(J)','flux(H)','flux(K)')

cS.add_votable_fields('sptype')

cS.add_votable_fields('otype(S)', 'otypes')

cS.add_votable_fields('parallax')

cS.add_votable_fields('propermotions')

cS.add_votable_fields('typed_id')

# cS.get_votable_fields()

In [9]:
table = cS.query_objects(df['name'][:])

/home/astrolander/.local/lib/python3.8/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 265 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : BD-22   924A 
  warnings.warn("Warning: The script line number %i raised "


## Предобработка таблицы (перевод в pandas и декодирование строк)

In [11]:
table

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,FLUX_U,FLUX_B,FLUX_V,FLUX_R,FLUX_I,FLUX_J,FLUX_H,FLUX_K,SP_TYPE,SP_QUAL,SP_BIBCODE,OTYPE_S,OTYPES,PLX_VALUE,PLX_PREC,PLX_ERROR,PLX_QUAL,PLX_BIBCODE,PMRA,PMDEC,PMRA_PREC,PMDEC_PREC,PM_ERR_MAJA,PM_ERR_MINA,PM_ERR_ANGLE,PM_QUAL,PM_BIBCODE,TYPED_ID
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,mag,mag,mag,mag,mag,mag,mag,mag,,,,,,mas,,mas,,,mas / yr,mas / yr,,,mas / yr,mas / yr,deg,,,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object,float32,float32,float32,float32,float32,float32,float32,float32,object,str1,object,object,object,float64,int16,float32,str1,object,float64,float64,int16,int16,float32,float32,int16,str1,object,object
HD 224820,00 00 55.3383,-30 03 51.003,14,14,0.068,0.038,90,A,O,2018yCat.1345....0G,--,8.43,8.41,--,--,8.343,8.41,8.36,A0V,C,1982MSS...C03....0H,Star,*|*|*iC|IR|UV,4.325,4,0.0747,A,2018yCat.1345....0G,18.495,2.429,3,3,0.108,0.077,90,A,2018yCat.1345....0G,HIP77
HD 225047,00 02 55.2272,-30 08 59.184,14,14,0.070,0.041,90,A,O,2018yCat.1345....0G,--,8.45,8.42,--,--,8.377,8.428,8.427,A0V,C,1982MSS...C03....0H,Star,*|*iC|V*|IR,4.184,4,0.0871,A,2018yCat.1345....0G,18.587,2.681,3,3,0.113,0.086,90,A,2018yCat.1345....0G,HIP232
HD 225077,00 03 12.7341,-31 12 44.581,14,14,0.158,0.088,90,A,O,2018yCat.1345....0G,--,8.93,8.88,--,--,8.705,8.724,8.66,A0V,C,1982MSS...C03....0H,SB*,SB*|SB*|*|*|*iC|IR,4.433,4,0.1536,A,2018yCat.1345....0G,18.420,3.530,2,2,1.100,0.590,0,A,2007A&A...474..653V,HIP257
HD 225102,00 03 25.6485,-46 49 51.515,14,14,0.036,0.035,90,A,O,2018yCat.1345....0G,--,11.27,11.2,--,--,10.972,10.969,10.976,A0/1V,D,1978MSS...C02....0H,Star,*|IR,0.997,4,0.056,A,2018yCat.1345....0G,-0.774,-2.586,3,3,0.064,0.066,90,A,2018yCat.1345....0G,HIP273
HD 225200,00 04 20.3172,-29 16 07.763,14,14,0.080,0.042,90,A,O,2018yCat.1345....0G,--,6.4,6.39,--,--,6.277,6.252,6.259,A1V,C,1982MSS...C03....0H,Star,*|*iC|IR|UV,7.437,4,0.0838,A,2018yCat.1345....0G,28.430,4.030,2,2,0.560,0.300,0,A,2007A&A...474..653V,HR9102
BD+61 2594,00 07 05.4869,+62 23 33.832,14,14,0.024,0.024,90,A,O,2018yCat.1345....0G,--,9.67,9.48,--,--,9.064,9.037,8.968,A0V,C,1972A&A....17..253M,Star,*|IR,2.886,4,0.0304,A,2018yCat.1345....0G,7.843,-1.937,3,3,0.048,0.041,90,A,2018yCat.1345....0G,HIP582
HD 325,00 07 37.5671,-71 48 43.669,14,14,0.034,0.026,90,A,O,2018yCat.1345....0G,--,9.8,9.72,--,--,9.609,9.59,9.594,A0V,C,1975MSS...C01....0H,Star,*|IR|UV,1.693,4,0.0348,A,2018yCat.1345....0G,21.443,-14.759,3,3,0.063,0.048,90,A,2018yCat.1345....0G,HD325
BD+62 15,00 11 37.4546,+63 26 35.285,14,14,0.029,0.027,90,A,O,2018yCat.1345....0G,--,9.36,9.2,--,--,8.902,8.937,8.9,A0V,C,1972A&A....17..253M,Star,*|IR|UV,1.677,4,0.037,A,2018yCat.1345....0G,4.617,-2.854,3,3,0.058,0.054,90,A,2016A&A...595A...2G,HIP942


In [25]:
pdtable = table.to_pandas()
# pdtable

In [26]:
pdtable.loc[0]

MAIN_ID                     b'HD 224820'
RA                         00 00 55.3383
DEC                        -30 03 51.003
RA_PREC                               14
DEC_PREC                              14
COO_ERR_MAJA                      0.0685
COO_ERR_MINA                      0.0381
COO_ERR_ANGLE                         90
COO_QUAL                               A
COO_WAVELENGTH                         O
COO_BIBCODE       b'2018yCat.1345....0G'
FLUX_U                               NaN
FLUX_B                              8.43
FLUX_V                              8.41
FLUX_R                               NaN
FLUX_I                               NaN
FLUX_J                             8.343
FLUX_H                              8.41
FLUX_K                              8.36
SP_TYPE                           b'A0V'
SP_QUAL                                C
SP_BIBCODE        b'1982MSS...C03....0H'
OTYPE_S                          b'Star'
OTYPES                  b'*|*|*iC|IR|UV'
PLX_VALUE       

In [27]:
string_fields = ['MAIN_ID','COO_BIBCODE','SP_TYPE','SP_BIBCODE','OTYPE_S',
                 'OTYPES', 'PLX_BIBCODE','PM_BIBCODE','TYPED_ID']

# types_dict = dict(zip(string_fields, ['string']*len(string_fields)))

# pdtable = pdtable.astype(types_dict)
# pdtable.dtypes

In [28]:
for col_name in string_fields:
    pdtable[col_name] = pdtable[col_name].str.decode('UTF-8')
# pdtable

In [30]:
pdtable.to_csv('Simbad_raw.csv')
# pdtable

### Добавление информации о количестве соседних звёзд
<span style="color:red">Их нужно отдельно искать скриптом на Симбаде</span>.

In [37]:
df2 = pd.read_csv('./simbad_res2_p.txt', sep='\n', skipinitialspace=True,
                 lineterminator='|', names=['name', '25s', '1.5m', '1', '2'],
                  index_col=None)

In [38]:
df2 = df2.drop(['1','2'], axis=1)
# df2

In [39]:
# df2[(df2['25s']!=1) | (df2['1.5m']!=1)]

In [35]:
result = pdtable.merge(df2, how='left', left_on='TYPED_ID', right_on='name').drop('name', axis=1)
# result

In [40]:
result.to_csv("simbad_full.csv", index=False)